In [59]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
import surprise.accuracy

# Import Data User, Anime, UserAnime

In [60]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))

# Membobotkan Nilai Status

In [76]:
# ScoresDF
ScoresDF.loc[ScoresDF['animes_rated'] == 5]

,username,anime_id,my_score,my_status,animes_rated,status_bobot,scoreXstatus
69479,blankzen,30276,9,2,5,5,9.0
69480,blankzen,5680,7,2,5,5,7.0
69481,blankzen,15315,7,2,5,5,7.0
69482,blankzen,28171,9,2,5,5,9.0
69483,blankzen,11933,6,6,5,2,2.4
155800,KyoXTohru1,120,10,2,5,5,10.0
155801,KyoXTohru1,853,10,2,5,5,10.0
155802,KyoXTohru1,24,9,1,5,3,5.4
155803,KyoXTohru1,3457,8,3,5,2,3.2
155804,KyoXTohru1,482,7,3,5,2,2.8


In [62]:
conditions = [
    (ScoresDF['my_status'] == 2),
    (ScoresDF['my_status'] == 3),
    (ScoresDF['my_status'] == 6),
    (ScoresDF['my_status'] == 1),
    (ScoresDF['my_status'] == 4)
]

values = [5,2,2,3,1]

ScoresDF['status_bobot'] = np.select(conditions,values)

# ScoresDF['status_bobot'] = np.where(ScoresDF['my_status'] == 2, 5, ScoresDF['my_status'])
# ScoresDF['status_bobot'] = np.where(ScoresDF['my_status'] == 3, 2, ScoresDF['my_status'])
# ScoresDF['status_bobot'] = np.where(ScoresDF['my_status'] == 6, 2, ScoresDF['my_status'])
# ScoresDF['status_bobot'] = np.where(ScoresDF['my_status'] == 1, 3, ScoresDF['my_status'])
# ScoresDF['status_bobot'] = np.where(ScoresDF['my_status'] == 4, 1, ScoresDF['my_status'])

In [63]:
ScoresDF.loc[ScoresDF['my_status'] == None]

,username,anime_id,my_score,my_status,animes_rated,status_bobot


# Mengkalikan Rating dengan Status

In [64]:
ScoresDF['scoreXstatus'] = ScoresDF['my_score'] * ScoresDF['status_bobot']
ScoresDF['scoreXstatus'] = ScoresDF['scoreXstatus'] / 5
ScoresDF

,username,anime_id,my_score,my_status,animes_rated,status_bobot,scoreXstatus
0,Tsundora,21,2,4,259,1,0.4
1,Tsundora,59,5,2,259,5,5.0
2,Tsundora,210,5,2,259,5,5.0
3,Tsundora,249,7,4,259,1,1.4
4,Tsundora,269,6,2,259,5,6.0
...,...,...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4,5,10.0
193268,Senpai_Helix,18677,9,2,4,5,9.0
193269,Kiyoshi-Kun,1722,10,2,3,5,10.0
193270,Kiyoshi-Kun,2993,10,2,3,5,10.0


# Membuat User-Item Matriks Berdasarkan Rating
Serta mengisi data yang kosong dengan 0

In [65]:
UserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='scoreXstatus')
UserItemRatingMatrix = UserItemRatingMatrix.replace(np.nan,0)
UserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training dan Test Model
Training dan test model dipisah 80% : 20%

In [66]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','scoreXstatus']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

In [67]:
sim_options = {'name': 'pearson',
                'min_support': 1
               }
algo_knn = KNNBasic(k=40, sim_options=sim_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Computing the pearson similarity matrix...


d:\Python\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in sqrt
  sim = construction_func[name](*args)


Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.202972926368368, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.305482989237236, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=7.707358535472425, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.700171719365383, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=6.522648451937305, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.0205377441373775, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=8.280047126783222, details={'actual_k': 33, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=8.361478435739782, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='Va

# Mendapatkan List Rekomendasi

In [98]:
# get the list of the movie ids
unique_ids = ScoresDF['anime_id'].unique()

# get the list of the ids that the username aleigh has rated
iidsranmaSA = ScoresDF.loc[ScoresDF['username']=="ranmaSA", 'anime_id']
iidsaleigh = ScoresDF.loc[ScoresDF['username']=="aleigh", 'anime_id']
iidsUky = ScoresDF.loc[ScoresDF['username']=="Uky", 'anime_id']
# remove the rated anime for the recommendations
anime_to_predict_aleigh = np.setdiff1d(unique_ids,iidsaleigh)
anime_to_predict_ranmaSA = np.setdiff1d(unique_ids,iidsranmaSA)
anime_to_predict_Uky = np.setdiff1d(unique_ids,iidsUky)
anime_to_predict = np.setdiff1d(anime_to_predict_aleigh,anime_to_predict_ranmaSA)
anime_to_predict = np.setdiff1d(anime_to_predict_Uky,anime_to_predict)

In [102]:
ranmasa_recs = []
for iid in anime_to_predict:
    ranmasa_recs.append((iid, algo_knn.predict(uid='ranmaSA', iid=iid).est))

ranmasa = pd.DataFrame(ranmasa_recs, columns=['iid', 'predict']).sort_values('predict', ascending=False)

In [103]:
aleigh_recs = []
for iid in anime_to_predict:
    aleigh_recs.append((iid, algo_knn.predict(uid='aleigh', iid=iid).est))

aleigh = pd.DataFrame(aleigh_recs, columns=['iid', 'predict']).sort_values('predict', ascending=False)

In [105]:
Uky_recs = []
for iid in anime_to_predict:
    Uky_recs.append((iid, algo_knn.predict(uid='Uky', iid=iid).est))

Uky = pd.DataFrame(Uky_recs, columns=['iid', 'predict']).sort_values('predict', ascending=False)

In [72]:
aleigh_status = ScoresDF[ScoresDF['username'] == "aleigh"]
aleigh_status = aleigh_status['my_status'].value_counts()
# print(aleigh_status)

ranmasa_status = ScoresDF[ScoresDF['username'] == "ranmaSA"]
ranmasa_status = ranmasa_status['my_status'].value_counts()
# print(ranmasa_status)

Skullio_status = ScoresDF[ScoresDF['username'] == "Skullio"]
Skullio_status = Skullio_status['my_status'].value_counts()
print(Skullio_status)

2    2
3    1
Name: my_status, dtype: int64


In [88]:
# testset_df = []
testset_df = pd.DataFrame(testset, columns=['username', 'anime_id','my_score'])
# testset_df.index = testset_df.Username

In [94]:
testset_df = testset_df['username'].value_counts().reset_index().rename(columns={"username": "animes_rated", "index": "username"})
testset_df

,username,animes_rated
0,Shouichirou,675
1,Hota-chan,488
2,xacthashtag,478
3,ziomalos,421
4,Takala,382
...,...,...
1027,Vinscent,1
1028,animeemo,1
1029,keji,1
1030,WarriorOfDestiny,1


In [ ]:
# testset_df
# testset_df.loc[testset_df['animes_rated'] == 3]

In [73]:
# prediction_df = pd.DataFrame(prediction_knn)
# print(prediction_df)

               uid    iid  r_ui       est  \
0      Shouichirou  12679   8.0  7.202973   
1           aleigh   2904   9.0  9.305483   
2        bigkfc2k8    267   6.0  7.707359   
3            Torah   6956  10.0  7.700172   
4             godT     63   7.0  6.522648   
...            ...    ...   ...       ...   
38650  Shouichirou    448   7.0  6.908410   
38651      mat3osz  11887  10.0  7.516927   
38652   Marlin_230  34618   8.0  7.042690   
38653    Morakillo  11319   8.0  7.589791   
38654       Relama  31404   6.0  5.898714   

                                         details  
0      {'actual_k': 32, 'was_impossible': False}  
1      {'actual_k': 40, 'was_impossible': False}  
2      {'actual_k': 40, 'was_impossible': False}  
3      {'actual_k': 40, 'was_impossible': False}  
4      {'actual_k': 40, 'was_impossible': False}  
...                                          ...  
38650  {'actual_k': 21, 'was_impossible': False}  
38651  {'actual_k': 40, 'was_impossible': False}  


In [74]:
surprise.accuracy.mae(prediction_knn)
surprise.accuracy.rmse(prediction_knn)

MAE:  1.4775
RMSE: 1.9834


1.9834231529616737